In [ ]:
import stellargraph as sg
from stellargraph import StellarGraph
from stellargraph.data import EdgeSplitter
from stellargraph.mapper import FullBatchLinkGenerator
from stellargraph.layer import GCN, LinkEmbedding
from stellargraph import datasets


from tensorflow import keras
from sklearn import preprocessing, feature_extraction, model_selection

from stellargraph import globalvar
from stellargraph import datasets
from IPython.display import display, HTML
%matplotlib inline
from IPython.display import clear_output

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm
import math
import sqlite3

In [ ]:
def analyze_general_metrics(byte_array, row_data):
    unigram_dist_int_chars = [0] * 256
    hamming_ones = 0
    hamming_total_bits = 0
    mean_byte_sum_byte_values = 0
    longest_streak = 0
    for byte in byte_array:
        ### unigram dist ###
        unigram_dist_int_chars[int(byte.hex(), 16)] += 1
        
        ### hamming weight ###
        bits = bin(int(byte.hex(), 16))[2:]
        hamming_total_bits += len(bits)
        for char in bits:
            if char == '1':
                hamming_ones += 1
                
        ### mean byte value ###
        # convert byte to hex then into int
        mean_byte_sum_byte_values += int(byte.hex(), 16)
                
        ### Longest Streak ###
        i = byte_array.index(byte)
        j = 0
        temp_streak = 1
        while i + j + 1 < len(byte_array) and byte_array[i + j] == byte_array[i + j + 1]:
            temp_streak += 1
            j += 1
        if temp_streak > longest_streak:
            longest_streak = temp_streak
    
    row_data["unigram_dist_mean"], row_data["unigram_dist_std"]  = norm.fit(unigram_dist_int_chars)
    row_data["hamming_weight"] = hamming_ones/hamming_total_bits
    row_data["mean_byte_value"] = mean_byte_sum_byte_values/len(byte_array)
    row_data["longest_streak"] = longest_streak
    row_data["file_size"] = len(byte_array)
    return row_data

In [ ]:
def getTables(cursor):
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    return cursor.fetchall()

In [ ]:
def analyzeCols(cols, table, conn, row_data):
    
    for col in cols:
        query = "SELECT " + col + " FROM " + table + ';'
        df = pd.read_sql_query(query, conn)
        if col != 'id' and df.dtypes[0] in ["int64", "float64"]:
            row_data["num_of_int_cols"] += 1
            row_data[table + "_" + col + "_mean"] = df.mean()[0]
            row_data[table + "_" + col + "_std"] = df.std()[0]
        elif col != 'id':
            r#ow_data[table + "_" + col + "_avg_num_of_chars"] = 0
            row_data["num_of_text_cols"] += 1
            char_dist = [0] * 128
            for row in df.index:
                #row_data[table + "_" + col + "_avg_num_of_chars"] += len(df[col][row])
                chars = 0
                numeric = 0
                for char in df[col][row]:
                    if ord(char) <= 128:
                        char_dist[ord(char)] += 1
                    if char.isalpha():
                        chars += 1
                    elif char.isnumeric():
                        numeric += 1
            #row_data[table + "_" + col + "_char_dist_mean"] = norm.fit(char_dist)[0]
            #row_data[table + "_" + col + "_char_dist_std"] = norm.fit(char_dist)[1]
            #row_data[table + "_" + col + "_avg_num_of_chars"] = row_data[table + "_" + col + "_avg_num_of_chars"] / len(df.index)
    return row_data

In [ ]:
def analyzeTable(table, conn, row_data):
    query = "SELECT * FROM " + table + ';'
    
    df = pd.read_sql_query(query, conn)
    
    row_data["num_of_cols"] += len(df.columns)
    row_data["num_of_rows"] += len(df.index)
    row_data[table + "_rows"] = len(df.index)
    row_data[table + "_cols"] = len(df.columns)
    row_data = analyzeCols(df.keys(), table, conn, row_data)
    return row_data

In [ ]:
def analyzeDB(db_path, db_name, row_data):
    
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    tables = getTables(cursor)
    row_data["num_of_tables"] = len(tables)

    for table in tables:
        row_data = analyzeTable(str(table)[2:-3], conn, row_data)
    
    return row_data

In [ ]:
def train_gcn(sdg, train_size, test_size, epochs_input, layer_size_num):
    
    edge_splitter_test = EdgeSplitter(sdg)

    G_test, edge_ids_test, edge_labels_test = edge_splitter_test.train_test_split(
        p=test_size, method="global", keep_connected=False
    )
    
    edge_splitter_train = EdgeSplitter(G_test)

    G_train, edge_ids_train, edge_labels_train = edge_splitter_train.train_test_split(
        p=train_size, method="global", keep_connected=False
    )
    
    epochs = epochs_input
    
    train_gen = FullBatchLinkGenerator(G_train, method="gcn")
    train_flow = train_gen.flow(edge_ids_train, edge_labels_train)
    
    test_gen = FullBatchLinkGenerator(G_test, method="gcn")
    test_flow = train_gen.flow(edge_ids_test, edge_labels_test)
    
    gcn = GCN(
        layer_sizes=[layer_size_num, layer_size_num], activations=["relu", "relu"], generator=train_gen, dropout=0.3
    )
    
    x_inp, x_out = gcn.in_out_tensors()
    
    prediction = LinkEmbedding(activation="relu", method="ip")(x_out)
    
    prediction = keras.layers.Reshape((-1,))(prediction)
    
    model = keras.Model(inputs=x_inp, outputs=prediction)

    model.compile(
        optimizer=keras.optimizers.Adam(lr=0.01),
        loss=keras.losses.binary_crossentropy,
        metrics=["binary_accuracy"],
    )
    
    model = keras.Model(inputs=x_inp, outputs=prediction)

    model.compile(
        optimizer=keras.optimizers.Adam(lr=0.01),
        loss=keras.losses.binary_crossentropy,
        metrics=["binary_accuracy"],
    )
    
    init_train_metrics = model.evaluate(train_flow)
    init_test_metrics = model.evaluate(test_flow)

    print("\nTrain Set Metrics of the initial (untrained) model:")
    for name, val in zip(model.metrics_names, init_train_metrics):
        print("\t{}: {:0.4f}".format(name, val))

    print("\nTest Set Metrics of the initial (untrained) model:")
    for name, val in zip(model.metrics_names, init_test_metrics):
        print("\t{}: {:0.4f}".format(name, val))
        
    history = model.fit(
        train_flow, epochs=epochs, validation_data=test_flow, verbose=2, shuffle=False
    )
    
    sg.utils.plot_history(history)
    
    train_metrics = model.evaluate(train_flow)
    test_metrics = model.evaluate(test_flow)

    print("\nTrain Set Metrics of the trained model:")
    for name, val in zip(model.metrics_names, train_metrics):
        print("\t{}: {:0.4f}".format(name, val))
        train_acc = val

    print("\nTest Set Metrics of the trained model:")
    for name, val in zip(model.metrics_names, test_metrics):
        print("\t{}: {:0.4f}".format(name, val))
    
    return train_metrics[1], train_metrics[0], test_metrics[1], test_metrics[0]

In [ ]:
num_of_dbs = 100
num_of_versions = 4
data = []
mapped_edges = []

for db_num in range(num_of_dbs):
    for version in range(num_of_versions):
        byte_array = []
        db_path = "demo_dbs_2/db_" + str(db_num) + "_v" + str(version) + ".sqlite"
        db_name = "db_" + str(db_num) + "_v" + str(version)
        row_data = {"db_name": db_name,
                    "unigram_dist_mean": 0,
                    "unigram_dist_std": 0,
                    "hamming_weight": 0,
                    "mean_byte_value": 0,
                    "longest_streak": 0,
                    "file_size": 0,
                    "num_of_tables": 0,
                    "num_of_cols": 0,
                    "num_of_rows": 0,
                    "num_of_text_cols": 0,
                    "num_of_int_cols": 0
                   }

        with open("demo_dbs_2/db_" + str(db_num) + "_v" + str(version) + ".sqlite", "rb") as f:
            byte = f.read(1)
            while byte:
                byte_array.append(byte)
                byte = f.read(1)
        row_data = analyze_general_metrics(byte_array, row_data)
        f.close()
        
        row_data = analyzeDB(db_path, db_name, row_data)
        
        data.append(row_data)
        
        if version < num_of_versions - 1:
            db_name_next = "db_" + str(db_num) + "_v" + str(version + 1)
            mapped_edges.append([db_name, db_name_next])
        clear_output(wait=True)
        print("Progress: " + str(((num_of_versions * db_num + version) / (num_of_dbs * num_of_versions)) * 100) + '%')
            
node_data = pd.DataFrame(data)
node_data.set_index("db_name", inplace=True)
node_data.index.name = None
node_data.fillna(0, inplace=True)
for column in node_data.columns:
    node_data[column] = node_data[column] /node_data[column].abs().max()
node_data.fillna(0, inplace=True)
node_data

In [ ]:
edges = pd.DataFrame(mapped_edges, columns=["source", "target"])
edges.head()

In [ ]:
sdg = StellarGraph({"db": node_data}, {"next": edges})
print(sdg.info())

In [ ]:
epochs = 50
results = pd.DataFrame([])
result_a = []
for layer_size in range (1, 3):
    for train_size in range(1, 10):
        for test_size in range(1, 10):
            train_acc, train_loss, test_acc, test_loss = train_gcn(sdg, train_size * 0.1, test_size * 0.1, epochs, layer_size * 16)
            result_a.append({
                'config': 'Layer Size: ' + str(layer_size * 16) + ' Train Size: ' + str(round(train_size * 0.1, 1)) + ' Test Size: ' + str(round(test_size * 0.1, 1)),
                'train_acc': train_acc,
                'train_loss': train_loss,
                'test_acc': test_acc,
                'test_loss': test_loss,
                'layer_size': layer_size,
                'train_size': train_size,
                'test_size': test_size
            })
            print('Progress: ' + str(len(result_a)/2) + '%')
results = pd.DataFrame(result_a)

In [ ]:
pd.set_option('display.max_rows', None)
results.sort_values(by=['test_acc'])

In [ ]:
best_ten_results = pd.DataFrame(results.sort_values(by=['test_acc']).iloc[-10:])
best_ten_results
avg_results_a = []
for index, row in best_ten_results.iterrows():
    train_acc_a = []
    train_loss_a = []
    test_acc_a = []
    test_loss_a = []
    for i in range(0, 10):
        train_acc, train_loss, test_acc, test_loss = train_gcn(sdg, row['train_size'] * 0.1, row['test_size'] * 0.1, 50, row['layer_size'] * 16)
        train_acc_a.append(train_acc)
        train_loss_a.append(train_loss)
        test_acc_a.append(test_acc)
        test_loss_a.append(test_loss)
    print(train_acc_a)
    avg_results_a.append({
        'config': 'Layer Size: ' + str(row['layer_size'] * 16) + ' Train Size: ' + str(round(row['train_size'] * 0.1, 1)) + ' Test Size: ' + str(round(row['test_size'] * 0.1, 1)),
        'Layer Size': row['layer_size'] * 16,
        'Train Size': round(row['train_size'] * 0.1, 1),
        'Test Size': round(row['test_size'] * 0.1, 1),
        'train_acc': norm.fit(train_acc_a)[0],
        'train_loss': norm.fit(train_loss_a)[0],
        'test_acc': norm.fit(test_acc_a)[0],
        'test_loss': norm.fit(test_loss_a)[0]       
    })
avg_results = pd.DataFrame(avg_results_a)

In [ ]:
avg_results

In [ ]:
avg_results.to_excel("output-sqlite.xlsx")